# 🎟️ Proyecto de Análisis de Marketing y Comportamiento de Usuarios en Showz
📊 Resumen del proyecto
Este análisis de datos se centra en el comportamiento de los usuarios y el rendimiento del marketing de Showz, una plataforma de venta de entradas para eventos. El objetivo es ofrecer recomendaciones prácticas para optimizar el presupuesto de marketing y mejorar las estrategias de adquisición de clientes.

🧩 El proyecto incluye:

🧠 Análisis del comportamiento del usuario:
Frecuencia de uso, duración de sesiones y patrones de regreso al sitio.

💸 Análisis de ventas e ingresos:
Tiempos de conversión, tamaño promedio de compra y valor de vida del cliente (LTV).

📈 Evaluación del rendimiento de marketing:
Cálculo de costos totales, CAC (costo de adquisición) por canal y ROMI.

🧪 Análisis de cohortes:
Comparación del rendimiento por fuente de tráfico y tipo de dispositivo.

📊 Visualizaciones:
Gráficos que muestran tendencias de visitas, compras y efectividad de campañas.

🧾 Herramientas utilizadas:
Python, pandas, matplotlib, seaborn, Jupyter Notebook

📍Objetivo:
Brindar recomendaciones estratégicas basadas en datos sobre dónde invertir, cuánto y por qué, para maximizar los resultados del marketing.

## Paso 1: Carga y procesamiento de datos.



In [1]:
#Primero cargamos las librerías necesarias para el proyecto.
import pandas as pd #Pandas para la manipulación de datos
import numpy as np 
import scipy.stats as st #Numpy y Scipy para operaciones matemáticas y estadísticas.
import matplotlib.pyplot as plt
import seaborn as sns #Matplotlib y seaborn para visualizaciones.

In [2]:
#Vamos a cargar los datos a continuación.
visits_df = pd.read_csv('../Data/visits_log_us.csv')
orders_df = pd.read_csv('../Data/orders_log_us.csv')
costs_df = pd.read_csv('../Data/costs_us.csv')

### Visits_df

In [3]:
#Ahora vamos a mostrar información general de los 3 datasets.
visits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


El DataFrame visits_df contiene información sobre visitas a la página web, incluyendo el dispositivo mediante el cual se inicio la sesión, la fecha y hora de inicio y final de la sesión, la fuente de la cuál provino ese cliente y el identificador único del usuario. Se puede ver que no hay valores ausentes, lo cual es un buen indicativo, voy a convertir las columnas de fechas al tipo de datos correspondientes, las demás son correctas, de igual manera vamos a estandarizar los nombres de las columnas a snake_case.

In [4]:
#Primero estandarizamos los nombres de las columnas.
visits_df.columns = visits_df.columns.str.lower().str.replace(' ', '_')
visits_df.columns


Index(['device', 'end_ts', 'source_id', 'start_ts', 'uid'], dtype='object')

In [5]:
#Convertimos las columnas start_ts y end_ts a tipo datetime.
visits_df['start_ts'] = pd.to_datetime(visits_df['start_ts'])
visits_df['end_ts'] = pd.to_datetime(visits_df['end_ts'])
visits_df.dtypes

device               object
end_ts       datetime64[ns]
source_id             int64
start_ts     datetime64[ns]
uid                  uint64
dtype: object

In [6]:
#Vamos a buscar valores duplicados.
visits_df.duplicated().sum()

0

Ahora es tiempo de optimizar los datos, para análisis posteriores. Por el momento vamos a agregar las columnas para mes y día de la semana y otra columna para la duración de la sesión.

In [7]:
#Extraemos la fecha de la sesión
visits_df['session_date'] = visits_df['start_ts'].dt.date.astype('datetime64[ns]')
visits_df['visit_year'] = visits_df['session_date'].dt.year
visits_df['session_month'] = visits_df['session_date'].dt.month
visits_df['session_dow'] = visits_df['session_date'].dt.day_of_week
#Calculamos la duración de la sesión en segundos.
visits_df['session_duration'] = visits_df['end_ts'] - visits_df['start_ts']
visits_df['session_duration'] = visits_df['session_duration'].dt.total_seconds()
visits_df.head()

,device,end_ts,source_id,start_ts,uid,session_date,visit_year,session_month,session_dow,session_duration
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062,2017-12-20,2017,12,2,1080.0
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740,2018-02-19,2018,2,0,1680.0
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527,2017-07-01,2017,7,5,0.0
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214,2018-05-20,2018,5,6,1440.0
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168,2017-12-27,2017,12,2,0.0


### Orders_df

In [8]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


El DataFrame orders_df contiene información de compras, incluyendo la fecha y hora de compra, el ingreso, y el user ID, los registros están completos. Vamos a estandarizar los nombres de las columnas y a convertir la columna buy_ts a tipo Datetime.

In [9]:
#Estandarizamos el nombre de las columnas.
orders_df.columns = orders_df.columns.str.lower().str.replace(' ', '_')
orders_df.columns

Index(['buy_ts', 'revenue', 'uid'], dtype='object')

In [10]:
#Convertimos a tipo Datetime
orders_df['buy_ts'] = pd.to_datetime(orders_df['buy_ts'])
orders_df.dtypes

buy_ts     datetime64[ns]
revenue           float64
uid                uint64
dtype: object

In [11]:
#Vamos a buscar duplicados.
orders_df.duplicated().sum()

0

Para este dataframe vamos a realizar lo mismo que con el otro excepto por la duración de la sesión, para posteriormente poder analizar y calcular las distintas métricas.

In [12]:
orders_df['order_date'] = orders_df['buy_ts'].dt.date.astype('datetime64[ns]')
orders_df['order_year'] = orders_df['buy_ts'].dt.year
orders_df['order_month'] = orders_df['order_date'].dt.month
orders_df['order_dow'] = orders_df['order_date'].dt.day_of_week
orders_df.head()

,buy_ts,revenue,uid,order_date,order_year,order_month,order_dow
0,2017-06-01 00:10:00,17.00,10329302124590727494,2017-06-01,2017,6,3
1,2017-06-01 00:25:00,0.55,11627257723692907447,2017-06-01,2017,6,3
2,2017-06-01 00:27:00,0.37,17903680561304213844,2017-06-01,2017,6,3
3,2017-06-01 00:29:00,0.55,16109239769442553005,2017-06-01,2017,6,3
4,2017-06-01 07:58:00,0.37,14200605875248379450,2017-06-01,2017,6,3


### Costs_df

In [13]:
costs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.7+ KB


El DataFrame costs_df contiene información sobre los gastos de Marketing, incluyendo la fuente en la que se invirtió(redes sociales, email, etc.), la fecha, y el total gastado en dicha fecha. Primero vamos a estandarizar el nombre de las columnas, después vamos a convertir a tipo datetime la columna dt, después vamos a verificar duplicados.

In [14]:
#Convertimos la columna a tipo Datetime
costs_df['dt'] = pd.to_datetime(costs_df['dt'])
costs_df.dtypes

source_id             int64
dt           datetime64[ns]
costs               float64
dtype: object

In [15]:
#Buscamos duplicados.
costs_df.duplicated().sum()

0

## Paso 2: Informes y métricas.

Para comenzar con nuestro análisis, calcularemos distintas métricas para dar respuesta a las siguientes preguntas con respecto a las visitas a la tienda de Showz:

* ¿Cuántas personas lo usan cada día, semana y mes?
* ¿Cuántas sesiones hay por día? (Un usuario puede tener más de una sesión).
* ¿Cuál es la duración de cada sesión?
* ¿Con qué frecuencia los usuarios regresan?



